### UC Berkeley, MICS, W202-Cryptography
### Week 06 Breakout 3
### Elgamal Elliptic Curve Cryptosystem

The Elgamal Elliptic Curve Cryptosystem is based on the computational intractability of the Elliptic Curve Discrete Logarithm Problem for a large prime number p.

You will find it very similar to the Elgamal Cryptosystem based on the Discrete Logarithm Problem we previously studied.  The main difference is that instead of exponents, we will use scalar multipliers for points on an elliptic curve prime curve.

You will also find it very similar to the Diffie Hellman Elliptic Curve Key Exchange, except that a chosen message may be sent, HOWEVER, note that the chosen message is a point on the elliptic curve, not something very meaningful. In the regular Elgamal, we can make the message any integer, and the message is typically a 256-bit key for AES256.  Since the chosen message is not the meanful, there is little advantage in using Elgamal Elliptic Curve over Diffie Hellman Elliptic Curve.

Since Elgamal Elliptic Curve is so similar to Elgamal, it's not a surprise that Elgamal Elliptic Curve is also subject to man in the middle attacks if there is no proper authentication of Alice and Bob.

The Elgamal Elliptic Curve algorithm works as follows (using the names and notation from our lectures):

* Publically known: an Elliptic Curve prime curve E in modulo prime p.  A, B, p, P are pubically known.

* Elgamal specifies that a "referee" controls E, A, B, p, P.

* Bob generates a large random number x of similar bit size to p that is < p.  The number can be odd or even, prime or composite.  This is Bob's scalar multiplier, which he must keep secret!

* Bob takes his scalar multiplier x and calculates Q = xP

* Bob sends Q to Alice

* Alice should validate the point Q she receives from Bob to guard against an invalid curve attack.

* Alice generates a large random number k of similar bit size to p that is < p.  The number can be odd or even, prime or composite.  This is Alice's scalar multiplier, which she must keep secret!

* Alice takes her secret scalar multiplier k and calculates A = kP

* Alice also gets to choose a message M (plaintext).  M must be a valid point.  (Since M is a point on an elliptic curve, with x and y values, it's not really of that much value to get to choose M.)

* Alice takes her secret scalar multiplier k, her message M (plaintext point), and the Q she recieved from Bob and calculates B (ciphertext of M) as B = M + kQ

* Alice sends her A and B to Bob.

* Bob should validate the points A and B he receives from Alice to guard against an invalid curve attack.

* Bob can now take the B (cipher text point) and decrypt it to the original message M (plaintext point) using Alice's A and B and his secret secret scalar multiplier x

* Since M is a point an on an elliptic curve, and we typically want to use it as a key for AES256 symmetrical encryption with a 256 bit key, it's common to concatenate the x and y values of the point and take a SHA256 hash.

In [1]:
import hashlib
from sage.all import *

In [2]:
def my_print_number(label, x):
    "prints a number in decimal, number of digits, hex, number of bits"
    
    print ("\n", label, '\n')
    print ("decimal:", "{:,}".format(x), "\n")
    print ("number of digits:", x.ndigits(), "\n")
    print ("hex:", x.hex(), "\n")
    print ("number of bits:", x.nbits(), "\n")

In [3]:
def my_random_k(p):
    "Given prime p, determine it's bit size, generate a random number 1 bit less than p's size"
    
    upper_limit = (p-2)
    lower_limit = ( (p-2) // 2)
    
    return Integer(randint(lower_limit, upper_limit))

#### We start with a publically known Elliptic Curve Prime Curve in modulo p.  A, B, p, P are publically known

In [4]:
# FIPS Curve P-256

p = 0xffffffff00000001000000000000000000000000ffffffffffffffffffffffff

A = -3
B = 0x5ac635d8aa3a93e7b3ebbd55769886bc651d06b0cc53b0f63bce3c3e27d2604b

P_x = 0x6b17d1f2e12c4247f8bce6e563a440f277037d812deb33a0f4a13945d898c296
P_y = 0x4fe342e2fe1a7f9b8ee7eb4a7c0f9e162bce33576b315ececbb6406837bf51f5

E = EllipticCurve(GF(p), [A,B])

P = E(P_x,P_y)

In [5]:
my_print_number("A", A)


 A 

decimal: -3 

number of digits: 1 

hex: -3 

number of bits: 2 



In [6]:
my_print_number("B", B)


 B 

decimal: 41,058,363,725,152,142,129,326,129,780,047,268,409,114,441,015,993,725,554,835,256,314,039,467,401,291 

number of digits: 77 

hex: 5ac635d8aa3a93e7b3ebbd55769886bc651d06b0cc53b0f63bce3c3e27d2604b 

number of bits: 255 



In [7]:
my_print_number("p", p)


 p 

decimal: 115,792,089,210,356,248,762,697,446,949,407,573,530,086,143,415,290,314,195,533,631,308,867,097,853,951 

number of digits: 78 

hex: ffffffff00000001000000000000000000000000ffffffffffffffffffffffff 

number of bits: 256 



In [8]:
my_print_number("Px", P_x)


 Px 

decimal: 48,439,561,293,906,451,759,052,585,252,797,914,202,762,949,526,041,747,995,844,080,717,082,404,635,286 

number of digits: 77 

hex: 6b17d1f2e12c4247f8bce6e563a440f277037d812deb33a0f4a13945d898c296 

number of bits: 255 



In [9]:
my_print_number("Py", P_y)


 Py 

decimal: 36,134,250,956,749,795,798,585,127,919,587,881,956,611,106,672,985,015,071,877,198,253,568,414,405,109 

number of digits: 77 

hex: 4fe342e2fe1a7f9b8ee7eb4a7c0f9e162bce33576b315ececbb6406837bf51f5 

number of bits: 255 



#### Bob generates a large random number x of similar bit size to p that is < p.  The number can be odd or even, prime or composite.  This is Bob's scalar multiplier, which he must keep secret!

In [10]:
bob_x = my_random_k(p)

In [11]:
my_print_number("Bob's x", bob_x)


 Bob's x 

decimal: 82,385,405,648,456,419,884,731,802,157,437,573,363,188,070,594,482,244,287,059,268,407,343,552,914,417 

number of digits: 77 

hex: b6247cb594d1e620a5adbf02889a965ea2ee351fba47c3279cef1df97cffe3f1 

number of bits: 256 



#### Bob takes his scalar multiplier x and calculates Q = xP, and sends Q to Alice

In [12]:
bob_Q = bob_x * P

In [13]:
my_print_number("Bob sends Alice - x value of point Q", Integer(bob_Q[0]))


 Bob sends Alice - x value of point Q 

decimal: 31,910,849,635,185,457,937,612,523,374,368,402,599,209,291,589,157,974,957,650,210,527,757,452,634,592 

number of digits: 77 

hex: 468ce69d67a86a802280d7d33063c1fdeb51781d3940bfe8daa0f2119c5ae9e0 

number of bits: 255 



In [14]:
my_print_number("Bob sends Alice - y value of point Q", Integer(bob_Q[1]))


 Bob sends Alice - y value of point Q 

decimal: 8,912,615,942,371,526,740,980,637,629,360,708,989,477,639,908,119,273,645,572,145,448,115,959,083,733 

number of digits: 76 

hex: 13b45ca2b8ba46c85cdce4495c34fe0be1d59431034f82c2cbc12517ea8db2d5 

number of bits: 253 



#### Alice generates a large random number k of similar bit size to p that is < p.  The number can be odd or even, prime or composite.  This is Alice's scalar multiplier, which she must keep secret!

In [15]:
alice_k = my_random_k(p)

In [16]:
my_print_number("Alice's k", alice_k)


 Alice's k 

decimal: 92,554,434,226,012,426,949,997,183,918,365,430,996,583,009,086,491,971,405,271,630,999,930,138,964,872 

number of digits: 77 

hex: cc9ff40f0b9204b82ff976b03ad0629d19d6a6da79e949a58b1552ed6a36b388 

number of bits: 256 



#### Alice takes her secret scalar multiplier k and calculates A = kP

In [17]:
alice_A = alice_k * P

In [18]:
my_print_number("Alice sends Bob - x value of point A", Integer(alice_A[0]))


 Alice sends Bob - x value of point A 

decimal: 66,918,875,690,487,395,372,282,539,396,473,561,393,244,230,821,497,368,899,466,550,450,430,925,791,413 

number of digits: 77 

hex: 93f2bdcd151a01fba55ee74b693c3e41b4c59e0e5b336a7422536974b9b2e0b5 

number of bits: 256 



In [19]:
my_print_number("Alice sends Bob - y value of point A", Integer(alice_A[1]))


 Alice sends Bob - y value of point A 

decimal: 74,994,365,150,417,712,493,719,428,275,367,454,043,505,696,240,152,259,002,767,456,502,124,112,482,071 

number of digits: 77 

hex: a5cd4e74d40a0457566e43d7eec2a41843649fc974a63e213f339f8147cb8f17 

number of bits: 256 



#### Alice also gets to choose a message M (plaintext).  M must be a valid point.  (Since M is a point on an elliptic curve, with x and y values, it's not really of that much value to get to choose M.)

In [20]:
alice_Mx = 0x6f09a38c6bc23c63c2871a56a047f0de1fcd125ada9380a3dc6b709c4ae53da2 
alice_My = 0x5b88e5be70c44b879f86211fe94b4e0f34b3797be3f636e2d9f37018b7758180

alice_M = E(alice_Mx, alice_My)

In [21]:
my_print_number("Alice's M - x value of point M", Integer(alice_M[0]))


 Alice's M - x value of point M 

decimal: 50,223,756,586,720,954,053,443,385,833,957,651,952,577,406,926,174,770,560,329,514,224,544,729,349,538 

number of digits: 77 

hex: 6f09a38c6bc23c63c2871a56a047f0de1fcd125ada9380a3dc6b709c4ae53da2 

number of bits: 255 



In [22]:
my_print_number("Alice's M - y value of point M", Integer(alice_M[1]))


 Alice's M - y value of point M 

decimal: 41,402,346,056,066,111,170,385,423,990,460,206,199,935,602,051,659,521,046,986,875,587,371,877,106,048 

number of digits: 77 

hex: 5b88e5be70c44b879f86211fe94b4e0f34b3797be3f636e2d9f37018b7758180 

number of bits: 255 



#### Alice takes her secret scalar multiplier k, her message M (plaintext point), and the Q she recieved from Bob and calculates B (ciphertext of M) as B = M + kQ

In [23]:
alice_B = alice_M + (alice_k * bob_Q)

In [24]:
my_print_number("Alice sends Bob - x value of point B", Integer(alice_B[0]))


 Alice sends Bob - x value of point B 

decimal: 106,870,557,817,215,340,324,471,603,036,077,850,205,339,723,524,759,017,397,487,010,669,780,398,900,405 

number of digits: 78 

hex: ec469797d481f91aa05fc2da3851e0bac2b9bed6fa7770082c3deb7e0cde04b5 

number of bits: 256 



In [25]:
my_print_number("Alice sends Bob - y value of point B", Integer(alice_B[1]))


 Alice sends Bob - y value of point B 

decimal: 78,888,388,263,440,235,280,532,158,931,158,101,616,872,271,167,658,436,443,093,962,117,989,686,040,928 

number of digits: 77 

hex: ae693ed5a54c889b65fb5278a1325184de42bcab3b603b92fe68110b2173d960 

number of bits: 256 



#### Alice sends her A and B to Bob.

#### Bob can now take the B (cipher text point) and decrypt it to the original message M (plaintext point) using Alice's A and B and his secret secret scalar multiplier x


In [26]:
bob_M = alice_B + (-bob_x * alice_A)

In [27]:
my_print_number("Bob's M - x value of point M", Integer(bob_M[0]))


 Bob's M - x value of point M 

decimal: 50,223,756,586,720,954,053,443,385,833,957,651,952,577,406,926,174,770,560,329,514,224,544,729,349,538 

number of digits: 77 

hex: 6f09a38c6bc23c63c2871a56a047f0de1fcd125ada9380a3dc6b709c4ae53da2 

number of bits: 255 



In [28]:
my_print_number("Bob's M - y value of point M", Integer(bob_M[1]))


 Bob's M - y value of point M 

decimal: 41,402,346,056,066,111,170,385,423,990,460,206,199,935,602,051,659,521,046,986,875,587,371,877,106,048 

number of digits: 77 

hex: 5b88e5be70c44b879f86211fe94b4e0f34b3797be3f636e2d9f37018b7758180 

number of bits: 255 



#### Both Alice and Bob should validate that the points they received from the other are indeed valid points on the same elliptic curve to guard against an invalid curve attack!

In [29]:
bob_Q.curve()

Elliptic Curve defined by y^2 = x^3 + 115792089210356248762697446949407573530086143415290314195533631308867097853948*x + 41058363725152142129326129780047268409114441015993725554835256314039467401291 over Finite Field of size 115792089210356248762697446949407573530086143415290314195533631308867097853951

In [30]:
alice_A.curve()

Elliptic Curve defined by y^2 = x^3 + 115792089210356248762697446949407573530086143415290314195533631308867097853948*x + 41058363725152142129326129780047268409114441015993725554835256314039467401291 over Finite Field of size 115792089210356248762697446949407573530086143415290314195533631308867097853951

In [31]:
alice_B.curve()

Elliptic Curve defined by y^2 = x^3 + 115792089210356248762697446949407573530086143415290314195533631308867097853948*x + 41058363725152142129326129780047268409114441015993725554835256314039467401291 over Finite Field of size 115792089210356248762697446949407573530086143415290314195533631308867097853951

In [32]:
if alice_A.curve() == bob_Q.curve() and alice_B.curve() == bob_Q.curve():
    print ("Both Alice's points and Bob's points are valid on the same curve")
else:
    print ("Warning - invalid curve attack!")

Both Alice's points and Bob's points are valid on the same curve


#### Validate that Alice's message and Bob's message match (remember messages are points on the elliptic curve)

In [33]:
if alice_M == bob_M:
    print ("Alice's message M matches Bob's message M")
else:
    print ("Alice's message M does NOT match Bob's message M")

Alice's message M matches Bob's message M


#### Since the message is a point on an elliptic curve, and we typically want to use it as a key for AES256 symmetrical encryption with a 256 bit key, it's common to concatenate the x and y values of the point and take a SHA256 hash.

In [34]:
alice_sha256 = hashlib.sha256((str(alice_M[0]) + str(alice_M[1])).encode('utf-8')).hexdigest()

bob_sha256 = hashlib.sha256((str(bob_M[0]) + str(bob_M[1])).encode('utf-8')).hexdigest()

print ("\nAlice and Bob should both have the same sha256 values for the message M chosen by Alice")

print ("Alice's sha256: " + alice_sha256)
print ("  Bob's sha256: " + bob_sha256)


Alice and Bob should both have the same sha256 values for the message M chosen by Alice
Alice's sha256: 356936999976905d6a4c5a506e548f4c6e8095f9720bc800314ed31c7db0ccb7
  Bob's sha256: 356936999976905d6a4c5a506e548f4c6e8095f9720bc800314ed31c7db0ccb7
